## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generate Recommended Actions based upon Customer Reviews using GenAI

### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- Use GenAI to a reccomended action to take for each customer review
- We want our LLM to determine the course of action to take for each customer review along with explaining its reasoning
    1. Loop through each customer review
    2. For reviews which do not already have a AI recommended action populated
    3. Generate our LLM prompt
    4. Execute our LLM prompt using BigFrames
    5. Update the customer review table with the AI recommended action in BigQuery

- This notebook will use GenAI to suggust the recommended action to take based upon the customer review
    - Example Actions:
        - "Thank the Customer"
        - "Send the customer a coupon"


- Notebook Logic:
    1. Gather the reviews that need a Recommended Action created.

    2. Create our LLM prompt:
        - The prompt will contain the review text.
        - The prompt will provide a list of recommended actions that the business can take.
            - We could also ask the LLM to generate new/interesting recommended actions
        - The prompt will ask the LLM to address any concerns within the review text.
        - The prompt will provide example JSON results so the LLM knows the output format to generate the results.

    3. Execute our LLM prompt using BigFrames

    4. Parse the JSON from the LLM and construct an UPDATE SQL statement so we can update the cutomer review table.
        - We check for valid JSON in case the LLM created incomplete or invalid JSON
        
    5. Execute the SQL statement against BigQuery.

## Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
#from bigframesllm import BigFramesLLM
from bigframes.ml.llm import GeminiTextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "${project_id}"
REGION = "us"
DATASET_ID = "${bigquery_data_beans_curated_dataset}"
CONNECTION_NAME = "vertex-ai"

connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
# bf.reset_session() # if you need to change the region
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
session = bf.get_global_session()

llm_model = GeminiTextGenerator(session=session, connection_name=connection)

## Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)


In [ ]:
# Runs a query against BigQuery and waits for it to complete
def RunQuery(sql):
  import time

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

## Generate Recommended Actions using GenAI

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
         WHERE gen_ai_recommended_action IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      You need to return JSON in the below "JSON format".
      Select one of the following actions based upon the below customer review who purchased coffee.
      - First randomly sort the actions.
      - Select the best action based upon the sentiment of the review.
      - It is okay to use the action "Send the customer a coupon" for both positive and negative reviews.
      - Return the results the below json format.
      - Do not include any special characters or "```json" in the json output

      Actions
      - "Thank the Customer"
      - "Apologize to the Customer"
      - "Send the customer a coupon"
      - "Call the customer"
      - "Promote Additional Products"
      - "Promise to Investigate"
      - "Encourage More Reviews"
      - "Invite Further Engagement"
      - "Reshare the review on other social media"

      JSON format: { "action" : "value", "explaination" : "llm explaination" }
      Sample JSON Response: { "action" : "Call the customer", "explaination" : "The customer left their phone number in the review." }
      Sample JSON Response: { "action" : "Encourage More Reviews", "explaination" : "Thanks for the review, please keep posting." }

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
                    SET gen_ai_recommended_action = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)